# Importing the Hackathon data 

In [1]:
import sys

In [2]:
import urllib.request

url = 'https://storage.googleapis.com/angostura-public/hult-hackathon-key.json'
urllib.request.urlretrieve(url, './hult-hackathon-key.json')

('./hult-hackathon-key.json', <http.client.HTTPMessage at 0x249197cbcf8>)

In [3]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './hult-hackathon-key.json'

In [4]:
 from google.cloud import bigquery

client = bigquery.Client()

In [5]:
QUERY = ('select * from `angostura_dev`.eh_health_survey_response')

df = client.query(QUERY).to_dataframe()

In [6]:
 print("Here are some quick details about the data:")
print(f">>  There are {len(df)} rows")
print(f">>  The shape of the df is {df.shape}")

Here are some quick details about the data:
>>  There are 1012 rows
>>  The shape of the df is (1012, 121)


In [7]:
 df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1012 entries, 0 to 1011
Columns: 121 entries, timestamp to power_outage_equipment_failure
dtypes: float64(10), int64(16), object(95)
memory usage: 956.7+ KB


In [8]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [9]:
 df.axes

[RangeIndex(start=0, stop=1012, step=1),
 Index(['timestamp', 'report_week', 'operability_icu', 'power_outage_equipment_failure_specify', 'power_generator_available', 'power_outage_mortatility', 'power_outage_deaths_count', 'mortality_hospital_failure_cardiovascular_count', 'mortality_hospital_failure_cardiovascular_cause', 'mortality_hospital_failure_trauma_count',
        ...
        'er_staff_nurse_night_on_call', 'er_staff_non_professional_nurse_night_on_call', 'wash_failure_icu', 'wash_failure_er', 'wahs_failure_sx', 'power_outage', 'power_outage_avg_failures_per_day', 'power_outage_days_count', 'power_outage_avg_duration', 'power_outage_equipment_failure'], dtype='object', length=121)]

In [10]:
df.head(10)

,timestamp,report_week,operability_icu,power_outage_equipment_failure_specify,power_generator_available,power_outage_mortatility,power_outage_deaths_count,mortality_hospital_failure_cardiovascular_count,mortality_hospital_failure_cardiovascular_cause,mortality_hospital_failure_trauma_count,mortality_hospital_failure_trauma_cause,strike_medical_staff_affected,strike_nurses_affected,operability_icu_p,strike_other_staff_affected,strike_patients_affected,strike_other_affected,staff_violence_affected_reasons,epidemiological_emergency_suspected_diseases,rrt_avail_high_flow_catheters,rrt_avail_blood_tests_hiv_hvb_hvc_vdr,rrt_avail_immediate_access_urea_reduction_bun,nCoV_face_mask_avail,nCoV_respiratory_isolation_protocol_avail,operability_er,nCoV_isolation_area_avail,operability_sx,operability_lab,operability_uls,operability_ct_mri,operability_xr,er_avail_adrenalin,er_avail_atropine,hospital_code,er_avail_dopamine,er_avail_cephalosporins_betalactams,er_avail_aminoglycosides_quinolone,er_avail_vancomycin_clindamycin,er_avail_lidocaine,er_avail_minor_opioids,er_avail_major_opioids,er_avail_iv_fluids,er_avail_diazepam_dph,er_avail_heparin,federal_entity,er_avail_steroids,er_avail_insulin,er_avail_asthma,er_avail_blood_pressure,er_avail_defibrillator,er_avail_ott_intubation,er_avail_catheter,er_avail_oxygen_suction,sx_avail_minor_opioids,sx_avail_major_opioids,hospital_type,sx_avail_anesthetic_gases,sx_avail_anesthetics_iv,sx_avail_relaxants,sx_avail_ott_intubation,sx_avail_patient_lingerie_kit,sx_avail_disposables_mask_gloves_gown,sx_avail_oxygen_suction,rrt_avail,rrt_operability,rrt_date_stopped_operability,administrative_entity,rrt_reason_stopped_operability,rrt_avg_daily_patients,rrt_peritoneal_count,rrt_peritoneal_reason_not_performed,rrt_num_hemodialysis,rrt_num_hemodialysis_acute,rrt_num_hemodialysis_chronic,rrt_num_hemodialysis_equipments,rrt_num_hemodialysis_equipments_operability,rrt_hemodialysis_avail_filter,arch_beds_count,rrt_hemodialysis_avail_lines,rrt_hemodialysis_avail_kit_hemodialysis,rrt_hemodialysis_avail_iron,rrt_hemodialysis_avail_b_complex,rrt_hemodialysis_avail_calcium,rrt_hemodialysis_avail_zemblar,rrt_reverse_osmosis_unit_operability,rrt_staff_nephrology,rrt_staff_md,rrt_staff_resident,op_beds_count,rrt_staff_nurse,rrt_staff_nurse_nephrologist,nutr_avail,nutr_operability,nutr_date_stopped_operability,nutr_reason_stopped_operability,nutr_num,nutr_daily_freq_meal,nutr_quality,nutr_freq_milk_formulas,op_beds_er_count,pneumonia_antibiotic_therapy_latency,mi_thrombolytic_treatment_latency,er_staff_residents_and_rural_day_on_call,er_staff_specialist_day_on_call,er_staff_mic_day_on_call,er_staff_nurse_day_on_call,er_staff_non_professional_nurse_day_on_call,er_staff_residents_and_rural_night_on_call,er_staff_specialist_night_on_call,er_staff_mic_night_on_call,op_pavilions_count,er_staff_nurse_night_on_call,er_staff_non_professional_nurse_night_on_call,wash_failure_icu,wash_failure_er,wahs_failure_sx,power_outage,power_outage_avg_failures_per_day,power_outage_days_count,power_outage_avg_duration,power_outage_equipment_failure
0,2019-08-25T02:24:44.452Z,34 del 2019,Todos los días,,Hay pero no funciona,No,0.0,2.0,"Equipos, Medicamentos, Recurso Humano, Resto e...",0.0,,,,Nunca ha existido,,,,,,,,,,,Todos los días,,Todos los días,Menos de 3 de días,No operativa,Nunca ha existido,No operativa,No hubo,No hubo,AMA000,No hubo,No hubo,No hubo,No hubo,Entre 3 y 5 días,No hubo,No hubo,No hubo,No hubo,No hubo,Amazonas,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,Entre 3 y 5 días,No hubo,No hubo,Tipo II,Todos los días,Todos los días,Todos los días,Todos los días,Todos los días,No hubo,Todos los días,Sí,No,Mas de 1 año,MPPS,"Falta de material quirúrgico, Falta de insumos...",NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,170,,,,,,,,,,,69,,,Sí,Sí,,,Entre 3 y 5 días,Se sirven menos de 3 comidas al día,No se siguen recomendaciones del especialista ...,No hubo fórmulas lácteas ningún día,36,0,0,1,4,2,3,2,1,2,2,01,2,1,"3 a 5 días , sin soporte

# Creating a sorted dataframe on 'hospital_code', 'Year', 'Week_No'

In [11]:
date_split = []
Viol = []
for i in range(len(df)):
    date_split.append(df['report_week'][i].split(' '))
    Viol.append(len(df['staff_violence_affected_reasons'][i]) != 0)
    
df['date_split'] = date_split
df['Viol'] = Viol

year = []
week_no = []
for i in range(len(df)):
    year.append(df['date_split'][i][2])
    week_no.append(df['date_split'][i][0])

df['Year'] = year
df['Week_No'] = week_no

df_sort = df.sort_values(by=['hospital_code', 'Year', 'Week_No'])

In [12]:
df_sort = df_sort.set_index([pd.Index(list(range(len(df_sort))))])

In [13]:
df_sort.head(2)

,timestamp,report_week,operability_icu,power_outage_equipment_failure_specify,power_generator_available,power_outage_mortatility,power_outage_deaths_count,mortality_hospital_failure_cardiovascular_count,mortality_hospital_failure_cardiovascular_cause,mortality_hospital_failure_trauma_count,mortality_hospital_failure_trauma_cause,strike_medical_staff_affected,strike_nurses_affected,operability_icu_p,strike_other_staff_affected,strike_patients_affected,strike_other_affected,staff_violence_affected_reasons,epidemiological_emergency_suspected_diseases,rrt_avail_high_flow_catheters,rrt_avail_blood_tests_hiv_hvb_hvc_vdr,rrt_avail_immediate_access_urea_reduction_bun,nCoV_face_mask_avail,nCoV_respiratory_isolation_protocol_avail,operability_er,nCoV_isolation_area_avail,operability_sx,operability_lab,operability_uls,operability_ct_mri,operability_xr,er_avail_adrenalin,er_avail_atropine,hospital_code,er_avail_dopamine,er_avail_cephalosporins_betalactams,er_avail_aminoglycosides_quinolone,er_avail_vancomycin_clindamycin,er_avail_lidocaine,er_avail_minor_opioids,er_avail_major_opioids,er_avail_iv_fluids,er_avail_diazepam_dph,er_avail_heparin,federal_entity,er_avail_steroids,er_avail_insulin,er_avail_asthma,er_avail_blood_pressure,er_avail_defibrillator,er_avail_ott_intubation,er_avail_catheter,er_avail_oxygen_suction,sx_avail_minor_opioids,sx_avail_major_opioids,hospital_type,sx_avail_anesthetic_gases,sx_avail_anesthetics_iv,sx_avail_relaxants,sx_avail_ott_intubation,sx_avail_patient_lingerie_kit,sx_avail_disposables_mask_gloves_gown,sx_avail_oxygen_suction,rrt_avail,rrt_operability,rrt_date_stopped_operability,administrative_entity,rrt_reason_stopped_operability,rrt_avg_daily_patients,rrt_peritoneal_count,rrt_peritoneal_reason_not_performed,rrt_num_hemodialysis,rrt_num_hemodialysis_acute,rrt_num_hemodialysis_chronic,rrt_num_hemodialysis_equipments,rrt_num_hemodialysis_equipments_operability,rrt_hemodialysis_avail_filter,arch_beds_count,rrt_hemodialysis_avail_lines,rrt_hemodialysis_avail_kit_hemodialysis,rrt_hemodialysis_avail_iron,rrt_hemodialysis_avail_b_complex,rrt_hemodialysis_avail_calcium,rrt_hemodialysis_avail_zemblar,rrt_reverse_osmosis_unit_operability,rrt_staff_nephrology,rrt_staff_md,rrt_staff_resident,op_beds_count,rrt_staff_nurse,rrt_staff_nurse_nephrologist,nutr_avail,nutr_operability,nutr_date_stopped_operability,nutr_reason_stopped_operability,nutr_num,nutr_daily_freq_meal,nutr_quality,nutr_freq_milk_formulas,op_beds_er_count,pneumonia_antibiotic_therapy_latency,mi_thrombolytic_treatment_latency,er_staff_residents_and_rural_day_on_call,er_staff_specialist_day_on_call,er_staff_mic_day_on_call,er_staff_nurse_day_on_call,er_staff_non_professional_nurse_day_on_call,er_staff_residents_and_rural_night_on_call,er_staff_specialist_night_on_call,er_staff_mic_night_on_call,op_pavilions_count,er_staff_nurse_night_on_call,er_staff_non_professional_nurse_night_on_call,wash_failure_icu,wash_failure_er,wahs_failure_sx,power_outage,power_outage_avg_failures_per_day,power_outage_days_count,power_outage_avg_duration,power_outage_equipment_failure,date_split,Viol,Year,Week_No
0,2019-06-14T21:14:15.397Z,22 del 2019,Todos los días,,Funciona menos de 3 días,No,NaN,1.0,"Equipos, Medicamentos, Recurso Humano",0.0,,,,Nunca ha existido,,,,"Robos , hurtos o disparos dentro del centro as...",,,,,,,Todos los días,,Todos los días,Menos de 3 de días,No operativa,Nunca ha existido,No operativa,No hubo,No hubo,AMA000,No hubo,Entre 3 y 5 días,No hubo,No hubo,Todos los días,Entre 1 y 2 días,No hubo,Entre 3 y 5 días,No hubo,No hubo,Amazonas,No hubo,No hubo,Entre 3 y 5 días,Entre 1 y 2 días,Todos los días,Todos los días,No hubo,Entre 3 y 5 días,Entre 3 y 5 días,No hubo,Tipo II,Todos los días,Todos los días,Todos los días,Todos los días,Todos los días,Todos los días,Todos los días,Sí,No,Mas de 1 año,MPPS,"Falta de material quirúrgico, Falta de insumos...",NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,170,,,,,,,,,,,83,,,Sí,Sí,,,Todos los días,Se sirven menos de 3 comidas al dí

# Creating a dictionary of hospital_code

In [14]:
# list of different hospital codes
hospital_dict = dict()
count = 1 

for i in df_sort['hospital_code']:
    if i in hospital_dict.keys():
        hospital_dict[i] = hospital_dict[i] + 1
        
    else:
        hospital_dict[i] = count
        

In [15]:
hospital_dict

{'AMA000': 37,
 'ANZ000': 37,
 'APU000': 28,
 'ARA001': 35,
 'ARA002': 9,
 'BAR000': 38,
 'BOL000': 51,
 'BOL001': 17,
 'CAR000': 29,
 'CAR001': 27,
 'DCA000': 37,
 'DCA001': 10,
 'DCA002': 34,
 'DCA003': 33,
 'DCA004': 33,
 'DCA005': 41,
 'DCA006': 38,
 'DCA007': 22,
 'DEL000': 39,
 'FAL000': 39,
 'GUA000': 38,
 'LAR000': 12,
 'MER000': 23,
 'MIR000': 32,
 'MIR001': 1,
 'MON000': 33,
 'NES000': 44,
 'POR000': 35,
 'SUC000': 38,
 'TAC000': 31,
 'TAC001': 31,
 'VAR000': 37,
 'ZUL000': 2,
 'ZUL001': 21}

# Creating a dictionsary of hospital_code and arch_beds_count

In [16]:
df_bed_hos = pd.DataFrame()

In [17]:
df_bed_hos['hospital_code'] = df_sort['hospital_code']
df_bed_hos['arch_beds_count'] = df_sort['arch_beds_count']

In [18]:
bed_count = df_bed_hos.drop_duplicates(subset='hospital_code', keep="last")

In [22]:
list(range(bed_count))

TypeError: 'DataFrame' object cannot be interpreted as an integer

In [23]:
bed_count['new_index'] = list(range(len(bed_count)))

C:\Users\pooji\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [24]:
bed_count = bed_count.set_index('new_index')

In [25]:
bed_count

,hospital_code,arch_beds_count
new_index,,
0,AMA000,170
1,ANZ000,320
2,APU000,200
3,ARA001,304
4,ARA002,500
5,BAR000,230
6,BOL000,470
7,BOL001,300
8,CAR000,456


In [26]:
bed_count_dict = dict()
for i in range(len(bed_count)):
    bed_count_dict[bed_count['hospital_code'][i]] = bed_count['arch_beds_count'][i]   

In [27]:
bed_count_dict

{'AMA000': 170,
 'ANZ000': 320,
 'APU000': 200,
 'ARA001': 304,
 'ARA002': 500,
 'BAR000': 230,
 'BOL000': 470,
 'BOL001': 300,
 'CAR000': 456,
 'CAR001': 456,
 'DCA000': 465,
 'DCA001': 400,
 'DCA002': 203,
 'DCA003': 1400,
 'DCA004': 515,
 'DCA005': 365,
 'DCA006': 470,
 'DCA007': 104,
 'DEL000': 300,
 'FAL000': 300,
 'GUA000': 160,
 'LAR000': 545,
 'MER000': 800,
 'MIR000': 600,
 'MIR001': 600,
 'MON000': 180,
 'NES000': 320,
 'POR000': 250,
 'SUC000': 366,
 'TAC000': 160,
 'TAC001': 900,
 'VAR000': 140,
 'ZUL000': 650,
 'ZUL001': 189}

# Create dataframe for data with violence

In [28]:
df_viol = df_sort[['report_week',
                   'hospital_code',
                   'strike_medical_staff_affected',
                   'strike_nurses_affected',
                   'strike_other_staff_affected',
                   'strike_patients_affected',
                   'strike_other_affected',
                   'staff_violence_affected_reasons',
                   'date_split',
                   'Year',
                   'Week_No',
                   'Viol']].copy()

In [29]:
df_viol.head(10)

,report_week,hospital_code,strike_medical_staff_affected,strike_nurses_affected,strike_other_staff_affected,strike_patients_affected,strike_other_affected,staff_violence_affected_reasons,date_split,Year,Week_No,Viol
0,22 del 2019,AMA000,,,,,,"Robos , hurtos o disparos dentro del centro as...","[22, del, 2019]",2019,22,True
1,23 del 2019,AMA000,,,,,Protestas sin suspensión de actividades,Violencia contra personal de hospital por fami...,"[23, del, 2019]",2019,23,True
2,24 del 2019,AMA000,,,,,,"Robos , hurtos o disparos dentro del centro as...","[24, del, 2019]",2019,24,True
3,25 del 2019,AMA000,,,,Protestas sin suspensión de actividades,,Violencia contra personal de hospital por fami...,"[25, del, 2019]",2019,25,True
4,26 del 2019,AMA000,,,,,,"Robos , hurtos o disparos dentro del centro as...","[26, del, 2019]",2019,26,True
5,27 del 2019,AMA000,,,,,,"Robos , hurtos o disparos dentro del centro as...","[27, del, 2019]",2019,27,True
6,28 del 2019,AMA000,,,,,,Violencia contra personal de hospital por fami...,"[28, del, 2019]",2019,28,True
7,29 del 2019,AMA000,,,,,,,"[29, del, 2019]",2019,29,False
8,30 del 2019,AMA000,,,,,,,"[30, del, 2019]",2019,30,False
9,31 del 2019,AMA000,,,,,,,"[31, del, 2019]",2019,31,False


# Question 10 --- Medicine Availability

In [30]:
df_medicine = df_sort[['report_week',
                      'hospital_code',
                      'arch_beds_count',
                      'er_avail_adrenalin',
                      'er_avail_atropine',
                      'er_avail_dopamine',
                      'er_avail_cephalosporins_betalactams',
                      'er_avail_aminoglycosides_quinolone',
                      'er_avail_vancomycin_clindamycin',
                      'er_avail_lidocaine',
                      'er_avail_minor_opioids',
                      'er_avail_major_opioids',
                      'er_avail_iv_fluids',
                      'er_avail_diazepam_dph',
                      'er_avail_heparin',
                      'er_avail_steroids',
                      'er_avail_insulin',
                      'er_avail_asthma',
                      'er_avail_blood_pressure',
                      'er_avail_defibrillator',
                      'er_avail_ott_intubation',
                      'er_avail_catheter',
                      'er_avail_oxygen_suction',
                      'date_split',
                      'Year',
                      'Week_No',
                      'Viol']].copy()

In [31]:
df_medicine.head(10)

,report_week,hospital_code,arch_beds_count,er_avail_adrenalin,er_avail_atropine,er_avail_dopamine,er_avail_cephalosporins_betalactams,er_avail_aminoglycosides_quinolone,er_avail_vancomycin_clindamycin,er_avail_lidocaine,er_avail_minor_opioids,er_avail_major_opioids,er_avail_iv_fluids,er_avail_diazepam_dph,er_avail_heparin,er_avail_steroids,er_avail_insulin,er_avail_asthma,er_avail_blood_pressure,er_avail_defibrillator,er_avail_ott_intubation,er_avail_catheter,er_avail_oxygen_suction,date_split,Year,Week_No,Viol
0,22 del 2019,AMA000,170,No hubo,No hubo,No hubo,Entre 3 y 5 días,No hubo,No hubo,Todos los días,Entre 1 y 2 días,No hubo,Entre 3 y 5 días,No hubo,No hubo,No hubo,No hubo,Entre 3 y 5 días,Entre 1 y 2 días,Todos los días,Todos los días,No hubo,Entre 3 y 5 días,"[22, del, 2019]",2019,22,True
1,23 del 2019,AMA000,170,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,Todos los días,Entre 1 y 2 días,No hubo,Entre 3 y 5 días,No hubo,Entre 1 y 2 días,No hubo,No hubo,Entre 3 y 5 días,No hubo,Todos los días,Todos los días,No hubo,Entre 1 y 2 días,"[23, del, 2019]",2019,23,True
2,24 del 2019,AMA000,170,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,Todos los días,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,Entre 1 y 2 días,Entre 3 y 5 días,No hubo,No hubo,No hubo,No hubo,Entre 1 y 2 días,"[24, del, 2019]",2019,24,True
3,25 del 2019,AMA000,170,Entre 1 y 2 días,No hubo,No hubo,Entre 1 y 2 días,No hubo,No hubo,Todos los días,No hubo,No hubo,No hubo,No hubo,Entre 1 y 2 días,No hubo,No hubo,Entre 1 y 2 días,No hubo,No hubo,No hubo,No hubo,Entre 3 y 5 días,"[25, del, 2019]",2019,25,True
4,26 del 2019,AMA000,170,No hubo,No hubo,No hubo,Entre 1 y 2 días,No hubo,No hubo,Todos los días,No hubo,No hubo,No hubo,No hubo,Entre 1 y 2 días,No hubo,Entre 1 y 2 días,No hubo,No hubo,No hubo,No hubo,No hubo,Entre 3 y 5 días,"[26, del, 2019]",2019,26,True
5,27 del 2019,AMA000,170,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,Entre 1 y 2 días,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,Entre 1 y 2 días,"[27, del, 2019]",2019,27,True
6,28 del 2019,AMA000,170,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,Entre 1 y 2 días,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,Entre 3 y 5 días,"[28, del, 2019]",2019,28,True
7,29 del 2019,AMA000,170,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,Entre 1 y 2 días,No hubo,No hubo,No hubo,Entre 1 y 2 días,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,Entre 3 y 5 días,"[29, del, 2019]",2019,29,False
8,30 del 2019,AMA000,170,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,Entre 1 y 2 días,No hubo,No hubo,No hubo,Entre 1 y 2 días,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,Entre 1 y 2 días,"[30, del, 2019]",2019,30,False
9,31 del 2019,AMA000,170,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,Entre 1 y 2 días,Entre 1 y 2 días,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,Entre 1 y 2 días,"[31, del, 2019]",2019,31,False


In [32]:
df_medicine_new = df_sort[['report_week',
                           'hospital_code',
                           'arch_beds_count',
                           'Year',
                           'Week_No',
                           'Viol']].copy()

In [33]:
def ConvertScale(column):
    a = []
    for i in df[column]:
        if i == 'No hubo':
            a.append(0)
        elif i == 'Entre 1 y 2 días':
            a.append(1/3)
        elif i == 'Entre 3 y 5 días':
            a.append(2/3)
        elif i == 'Todos los días':
            a.append(1)
        else:
            a.append('x')
    
    return a

In [34]:
def NewColumn(column):
    x = column.split('_')
    k = ''
    if len(x) > 3:
        k = x[2] + '_' + x[3]
    else:
        k = x[2]    
    return k

In [35]:
df_medicine_new[NewColumn('er_avail_adrenalin')] = ConvertScale('er_avail_adrenalin')
df_medicine_new[NewColumn('er_avail_atropine')] = ConvertScale('er_avail_atropine')
df_medicine_new[NewColumn('er_avail_dopamine')] = ConvertScale('er_avail_dopamine')
df_medicine_new[NewColumn('er_avail_cephalosporins_betalactams')] = ConvertScale('er_avail_cephalosporins_betalactams')
df_medicine_new[NewColumn('er_avail_aminoglycosides_quinolone')] = ConvertScale('er_avail_aminoglycosides_quinolone')
df_medicine_new[NewColumn('er_avail_vancomycin_clindamycin')] = ConvertScale('er_avail_vancomycin_clindamycin')
df_medicine_new[NewColumn('er_avail_lidocaine')] = ConvertScale('er_avail_lidocaine')
df_medicine_new[NewColumn('er_avail_minor_opioids')] = ConvertScale('er_avail_minor_opioids')
df_medicine_new[NewColumn('er_avail_major_opioids')] = ConvertScale('er_avail_major_opioids')
df_medicine_new[NewColumn('er_avail_iv_fluids')] = ConvertScale('er_avail_iv_fluids')
df_medicine_new[NewColumn('er_avail_diazepam_dph')] = ConvertScale('er_avail_diazepam_dph')
df_medicine_new[NewColumn('er_avail_heparin')] = ConvertScale('er_avail_heparin')
df_medicine_new[NewColumn('er_avail_steroids')] = ConvertScale('er_avail_steroids')
df_medicine_new[NewColumn('er_avail_insulin')] = ConvertScale('er_avail_insulin')
df_medicine_new[NewColumn('er_avail_asthma')] = ConvertScale('er_avail_asthma')
df_medicine_new[NewColumn('er_avail_blood_pressure')] = ConvertScale('er_avail_blood_pressure')
df_medicine_new[NewColumn('er_avail_defibrillator')] = ConvertScale('er_avail_defibrillator')
df_medicine_new[NewColumn('er_avail_ott_intubation')] = ConvertScale('er_avail_ott_intubation')
df_medicine_new[NewColumn('er_avail_catheter')] = ConvertScale('er_avail_catheter')
df_medicine_new[NewColumn('er_avail_oxygen_suction')] = ConvertScale('er_avail_oxygen_suction')

In [36]:
df_medicine_new.head(10)

,report_week,hospital_code,arch_beds_count,Year,Week_No,Viol,adrenalin,atropine,dopamine,cephalosporins_betalactams,aminoglycosides_quinolone,vancomycin_clindamycin,lidocaine,minor_opioids,major_opioids,iv_fluids,diazepam_dph,heparin,steroids,insulin,asthma,blood_pressure,defibrillator,ott_intubation,catheter,oxygen_suction
0,22 del 2019,AMA000,170,2019,22,True,0,0.000000,0.0,0.000000,0,0,0.666667,0.000000,0,0,0,0,0,0,0,0,0,0,0,0.666667
1,23 del 2019,AMA000,170,2019,23,True,0,0.000000,0.0,0.666667,0,0,0.666667,0.000000,0,0,0,0.666667,0,0,0,0,0,1,0,0.666667
2,24 del 2019,AMA000,170,2019,24,True,0,0.000000,0.0,0.000000,0,0,1.000000,0.000000,0,0,0,0,0,0.333333,0.666667,0,0,0,0,0.333333
3,25 del 2019,AMA000,170,2019,25,True,0,0.000000,0.0,0.000000,0,0,1.000000,0.000000,0,0.333333,0,0,0,0,0,0.333333,0,0,0,0
4,26 del 2019,AMA000,170,2019,26,True,0,0.000000,0.0,0.000000,0,0,0.333333,0.000000,0,0,0,0,0,0,0,0,0,0,0,0.333333
5,27 del 2019,AMA000,170,2019,27,True,0,0.000000,0.0,0.000000,0,0,1.000000,0.000000,0,0,0,0.666667,0,0,0,0.666667,0,0,0,0.333333
6,28 del 2019,AMA000,170,2019,28,True,0.333333,0.666667,0.0,0.333333,0,0,0.666667,0.333333,0,0.666667,0,0.666667,0,0,0.333333,0,0,0,0,0.666667
7,29 del 2019,AMA000,170,2019,29,False,0,0.000000,0.0,0.000000,0,0,0.000000,0.000000,0,0,0,0.333333,0,1,0,0,0,0,0,0
8,30 del 2019,AMA000,170,2019,30,False,0,0.000000,0.0,0.000000,0,0,1.000000,0.000000,0,0,0,0,0,0,0,0.666667,0,0,0,0.333333
9,31 del 2019,AMA000,170,2019,31,False,0,0.000000,0.0,0.000000,0,0,0.333333,0.333333,0,0,0,0,0,0,0,0,0,0,0,0.333333


# Defining the Availability and Reliability Metrics

In [37]:
def Remove_notavail(drug_avail):
    new_drug_avail = []
    for i in drug_avail:
        if i != 'x':
            new_drug_avail.append(i)
    
    return new_drug_avail
    

In [38]:
def AvailabilityM(drug_avail):
    
    new_drug_avail = Remove_notavail(drug_avail)
    if len(new_drug_avail) != 0:
        avail = sum(new_drug_avail)/ len(new_drug_avail)
    else:
        avail = 0
    
    return avail

In [39]:
def NationalAvailability(drug, begin_time, end_time):
    
    n_avail = []
    bed_avail = []
    for i in hospital_dict.keys():
        per_hospital = df_medicine_new.loc[df_sort['hospital_code'] == i]
        start_index = per_hospital.loc[per_hospital['report_week'] == begin_time].index.values
        end_index = per_hospital.loc[per_hospital['report_week'] == end_time].index.values + 1
        if len(start_index) != 0 and len(end_index) != 0:
            drug_avail = per_hospital[drug][start_index[0]:end_index[0]]
        else:
            drug_avail = [0]
            
        bed_avail.append(bed_count_dict[i])
            
        n_avail.append(AvailabilityM(drug_avail))
        
    n_avail_avg = []    
    for i in range(len(n_avail)):
        n_avail_avg.append(n_avail[i]*bed_avail[i])
        
    national_availability_index = sum(n_avail_avg)/sum(bed_avail)   
    
    return national_availability_index

In [40]:
def ReliabilityM(drug_avail):
    
    old = len(drug_avail)
    new_drug_avail = Remove_notavail(drug_avail)
    new = len(new_drug_avail)
    
    calc = old-new
    if old != 0:
        rely = 1 - (calc/old)
    else:
        rely = 0
    
    return rely

In [41]:
def NationalReliability(drug, begin_time, end_time):
    
    n_avail = []
    bed_avail = []
    for i in hospital_dict.keys():
        per_hospital = df_medicine_new.loc[df_sort['hospital_code'] == i]
        start_index = per_hospital.loc[per_hospital['report_week'] == begin_time].index.values
        end_index = per_hospital.loc[per_hospital['report_week'] == end_time].index.values + 1
        if len(start_index) != 0 and len(end_index) != 0:
            drug_avail = per_hospital[drug][start_index[0]:end_index[0]]
        else:
            drug_avail = [0]
            
        bed_avail.append(bed_count_dict[i])
            
        n_avail.append(ReliabilityM(drug_avail))
        
    n_avail_avg = []    
    for i in range(len(n_avail)):
        n_avail_avg.append(n_avail[i]*bed_avail[i])
        
    national_reliability_index = sum(n_avail_avg)/sum(bed_avail)   
    
    return national_reliability_index

In [42]:
def AvailDrug(Hcode, drug, begin_time, end_time):
    
    per_hospital = df_medicine_new.loc[df_sort['hospital_code'] == Hcode]
    start_index = per_hospital.loc[per_hospital['report_week'] == begin_time].index.values
    end_index = per_hospital.loc[per_hospital['report_week'] == end_time].index.values + 1

    drug_avail = per_hospital[drug][start_index[0]:end_index[0]]
    
    reliability = ReliabilityM(drug_avail)
    availability = AvailabilityM(drug_avail)
    
    national_availability_index = NationalAvailability(drug, begin_time, end_time)
    national_reliability_index = NationalReliability(drug, begin_time, end_time)
    
    return (reliability, availability, national_reliability_index, national_availability_index)

In [43]:
# pick a specific Hospital, drug and a window of inspection --- both dates inclusive
k1, l1, m1 , n1 = AvailDrug('AMA000', 'cephalosporins_betalactams', '28 del 2019', '38 del 2019')
print(k1) # relyability
print(l1) # availbility
print(m1) # national_reliability_index
print(n1) # national_availability_index

1.0
0.1818181818181818
0.4850594720046417
0.0022417385341667326


In [44]:
k2, l2, m2 , n2 = AvailDrug('AMA000', 'lidocaine', '28 del 2019', '38 del 2019')
print(k2) # relyability
print(l2) # availbility
print(m2) # national_reliability_index
print(n2) # national_availability_index

1.0
0.6363636363636364
0.4850594720046417
0.007846084869583565


In [45]:
k3, l3, m3 , n3 = AvailDrug('FAL000', 'cephalosporins_betalactams', '40 del 2019', '45 del 2019')
print(k3) # relyability
print(l3) # availbility
print(m3) # national_reliability_index
print(n3) # national_availability_index

0
0
0.42638526254714243
0.000587121997043668


# Question 11 --- Surgical Supplies

In [50]:
df_surgical = df_sort[['report_week',
                      'hospital_code',
                      'sx_avail_minor_opioids',
                      'sx_avail_major_opioids',
                      'sx_avail_anesthetic_gases',
                      'sx_avail_anesthetics_iv',
                      'sx_avail_relaxants',
                      'sx_avail_ott_intubation',
                      'sx_avail_patient_lingerie_kit',
                      'sx_avail_disposables_mask_gloves_gown',
                      'sx_avail_oxygen_suction',
                      'date_split',
                      'Year',
                      'Week_No']].copy()

In [51]:
df_surgical_new = df_sort[['report_week',
                           'hospital_code',
                           'Year',
                           'Week_No']].copy()

In [52]:
def ConvertScale(column):
    a = []
    for i in df[column]:
        if i == 'No hubo':
            a.append(0)
        elif i == 'Entre 1 y 2 días':
            a.append(1/3)
        elif i == 'Entre 3 y 5 días':
            a.append(2/3)
        elif i == 'Todos los días':
            a.append(1)
        else:
            a.append('x')
    
    return a

In [53]:
def NewColumn(column):
    x = column.split('_')
    k = ''
    if len(x) > 3:
        k = x[2] + '_' + x[3]
    else:
        k = x[2]    
    return k

In [54]:
df_surgical_new[NewColumn('sx_avail_minor_opioids')] = ConvertScale('sx_avail_minor_opioids')
df_surgical_new[NewColumn('sx_avail_major_opioids')] = ConvertScale('sx_avail_major_opioids')
df_surgical_new[NewColumn('sx_avail_anesthetic_gases')] = ConvertScale('sx_avail_anesthetic_gases')
df_surgical_new[NewColumn('sx_avail_anesthetics_iv')] = ConvertScale('sx_avail_anesthetics_iv')
df_surgical_new[NewColumn('sx_avail_relaxants')] = ConvertScale('sx_avail_relaxants')
df_surgical_new[NewColumn('sx_avail_ott_intubation')] = ConvertScale('sx_avail_ott_intubation')
df_surgical_new[NewColumn('sx_avail_patient_lingerie_kit')] = ConvertScale('sx_avail_patient_lingerie_kit')
df_surgical_new[NewColumn('sx_avail_disposables_mask_gloves_gown')] = ConvertScale('sx_avail_disposables_mask_gloves_gown')
df_surgical_new[NewColumn('sx_avail_oxygen_suction')] = ConvertScale('sx_avail_oxygen_suction')

In [56]:
df_surgical_new.head()

,report_week,hospital_code,Year,Week_No,minor_opioids,major_opioids,anesthetic_gases,anesthetics_iv,relaxants,ott_intubation,patient_lingerie,disposables_mask,oxygen_suction
0,22 del 2019,AMA000,2019,22,0.0,0,1.0,1.0,1,1,1,0,1.0
1,23 del 2019,AMA000,2019,23,0.0,0,1.0,1.0,1,1,0,0,1.0
2,24 del 2019,AMA000,2019,24,0.0,0,1.0,1.0,1,1,0.666667,0,1.0
3,25 del 2019,AMA000,2019,25,0.0,0,1.0,1.0,1,1,0.333333,0,1.0
4,26 del 2019,AMA000,2019,26,0.0,0,1.0,1.0,1,1,0.333333,0.666667,1.0


In [58]:
def Remove_notavail(supply_avail):
    new_supply_avail = []
    for i in supply_avail:
        if i != 'x':
            new_supply_avail.append(i)
    
    return new_supply_avail
    

def AvailabilityM(supply_avail):
    
    new_supply_avail = Remove_notavail(supply_avail)
    if len(new_supply_avail) != 0:
        avail = sum(new_supply_avail)/ len(new_supply_avail)
    else:
        avail = 0
    
    return avail

def NationalAvailability(supply, begin_time, end_time):
    
    n_avail = []
    bed_avail = []
    for i in hospital_dict.keys():
        per_hospital = df_surgical_new.loc[df_sort['hospital_code'] == i]
        start_index = per_hospital.loc[per_hospital['report_week'] == begin_time].index.values
        end_index = per_hospital.loc[per_hospital['report_week'] == end_time].index.values + 1
        if len(start_index) != 0 and len(end_index) != 0:
            supply_avail = per_hospital[supply][start_index[0]:end_index[0]]
        else:
            supply_avail = [0]
            
        bed_avail.append(bed_count_dict[i])
            
        n_avail.append(AvailabilityM(supply_avail))
        
    n_avail_avg = []    
    for i in range(len(n_avail)):
        n_avail_avg.append(n_avail[i]*bed_avail[i])
        
    national_availability_index = sum(n_avail_avg)/sum(bed_avail)   
    
    return national_availability_index

def ReliabilityM(supply_avail):
    
    old = len(supply_avail)
    new_supply_avail = Remove_notavail(supply_avail)
    new = len(new_supply_avail)
    
    calc = old-new
    if old != 0:
        rely = 1 - (calc/old)
    else:
        rely = 0
    
    return rely

def NationalReliability(supply, begin_time, end_time):
    
    n_avail = []
    bed_avail = []
    for i in hospital_dict.keys():
        per_hospital = df_surgical_new.loc[df_sort['hospital_code'] == i]
        start_index = per_hospital.loc[per_hospital['report_week'] == begin_time].index.values
        end_index = per_hospital.loc[per_hospital['report_week'] == end_time].index.values + 1
        if len(start_index) != 0 and len(end_index) != 0:
            supply_avail = per_hospital[supply][start_index[0]:end_index[0]]
        else:
            supply_avail = [0]
            
        bed_avail.append(bed_count_dict[i])
            
        n_avail.append(ReliabilityM(supply_avail))
        
    n_avail_avg = []    
    for i in range(len(n_avail)):
        n_avail_avg.append(n_avail[i]*bed_avail[i])
        
    national_reliability_index = sum(n_avail_avg)/sum(bed_avail)   
    
    return national_reliability_index

def AvailDrug(Hcode, supply, begin_time, end_time):
    
    per_hospital = df_surgical_new.loc[df_sort['hospital_code'] == Hcode]
    start_index = per_hospital.loc[per_hospital['report_week'] == begin_time].index.values
    end_index = per_hospital.loc[per_hospital['report_week'] == end_time].index.values + 1

    supply_avail = per_hospital[supply][start_index[0]:end_index[0]]
    
    reliability = ReliabilityM(supply_avail)
    availability = AvailabilityM(supply_avail)
    
    national_availability_index = NationalAvailability(supply, begin_time, end_time)
    national_reliability_index = NationalReliability(supply, begin_time, end_time)
    
    return (reliability, availability, national_reliability_index, national_availability_index)

In [60]:
# pick a specific Hospital, drug and a window of inspection --- both dates inclusive
k1, l1, m1 , n1 = AvailDrug('AMA000', 'anesthetic_gases', '28 del 2019', '38 del 2019')
print(k1) # relyability
print(l1) # availbility
print(m1) # national_reliability_index
print(n1) # national_availability_index

1.0
0.9696969696969698
0.4850594720046417
0.011955938848889243


In [61]:
k2, l2, m2 , n2 = AvailDrug('AMA000', 'disposables_mask', '28 del 2019', '38 del 2019')
print(k2) # relyability
print(l2) # availbility
print(m2) # national_reliability_index
print(n2) # national_availability_index

1.0
0.1212121212121212
0.4850594720046417
0.001494492356111155


In [62]:
k3, l3, m3 , n3 = AvailDrug('FAL000', 'anesthetic_gases', '40 del 2019', '45 del 2019')
print(k3) # relyability
print(l3) # availbility
print(m3) # national_reliability_index
print(n3) # national_availability_index

0
0
0.42638526254714243
0.01232956193791703


# Create data-frame for test

In [46]:
df_try = pd.DataFrame() 
df_try['report_week'] = ['26 del 2019',
                        '27 del 2019',
                        '28 del 2019',
                        '29 del 2019',
                        '30 del 2019',
                        '31 del 2019',
                        '32 del 2019',
                        '33 del 2019',
                        '34 del 2019',
                        '35 del 2019',
                        '36 del 2019',
                        '37 del 2019',
                        '38 del 2019',
                        '39 del 2019']

df_try['hospital_code'] = ['AMA000',
                        'AMA000',
                        'AMA000',
                        'AMA000',
                        'AMA000',
                        'AMA000',
                        'AMA000',
                        'AMA000',
                        'AMA000',
                        'AMA000',
                        'AMA000',
                        'AMA000',
                        'AMA000',
                        'AMA000']

df_try['cephalosporins_betalactams'] = [0.3,0.6,0.6,0.3,'x','x',0.3,0,0,0,'x',0.6,0.3,0]

In [47]:
df_try

,report_week,hospital_code,cephalosporins_betalactams
0,26 del 2019,AMA000,0.3
1,27 del 2019,AMA000,0.6
2,28 del 2019,AMA000,0.6
3,29 del 2019,AMA000,0.3
4,30 del 2019,AMA000,x
5,31 del 2019,AMA000,x
6,32 del 2019,AMA000,0.3
7,33 del 2019,AMA000,0
8,34 del 2019,AMA000,0
9,35 del 2019,AMA000,0


In [48]:
# end_time included
begin_time = '28 del 2019'
end_time = '38 del 2019'

start_index = df_try.loc[df_try['report_week'] == begin_time].index.values
end_index = df_try.loc[df_try['report_week'] == end_time].index.values + 1
drug_avail = df_try['cephalosporins_betalactams'][start_index[0]:end_index[0]]